In [1]:
import pandas as pd
df = pd.read_csv('data_bert1.csv')
df.head()

,category,text,clean_text,words_removed
0,arts,you are using an older browser version. please...,you are an older browser version . please use ...,50
1,arts,on 31 march two of classical music s most acco...,on 31 march two of classical music s most acco...,156
2,arts,bpt after a year of being locked away at home ...,after a year of being locked away at home the ...,98
3,arts,pilot uninjured plane hit sandbar while landin...,pilot uninjured plane hit while landing . a fl...,22
4,arts,colleen distin photo by facebook toronto sun ....,colleen photo by sun . a lost wallet been retu...,47


In [2]:
from sentence_transformers import SentenceTransformer

model_name = "all-mpnet-base-v2"
embedder = SentenceTransformer(model_name)


corpus = df["clean_text"].tolist()
embeddings = embedder.encode(corpus, show_progress_bar=True)


Batches:   0%|          | 0/166 [00:00<?, ?it/s]

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report

# Features (X) and labels (y)
X = embeddings
y = df["category"].values

# Split data into train/test (or train/val/test if you prefer)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

param_grid = {
    "n_estimators": [100, 200, 300],
    "max_depth": [None, 5, 10],
    "max_features": ['sqrt', 'log2'],
    "class_weight": ['balanced_subsample', 'balanced', None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    # Add more params if desired, e.g. "min_samples_split": [2, 5]
}

rf_clf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(
    rf_clf,
    param_grid=param_grid,
    cv=2,               # 3-fold cross validation
    scoring="accuracy", # or "accuracy", "f1_weighted", etc.
    n_jobs=-1,          # utilize all CPU cores
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best params:", grid_search.best_params_)
print("Best CV score (accuracy):", grid_search.best_score_)


best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

print("\nTEST SET PERFORMANCE:")
print(classification_report(y_test, y_pred, zero_division=0))

Fitting 2 folds for each of 486 candidates, totalling 972 fits


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Best params: {'class_weight': 'balanced_subsample', 'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 300}
Best CV score (accuracy): 0.4538140027663357

TEST SET PERFORMANCE:
               precision    recall  f1-score   support

         arts       0.33      0.43      0.37        58
        crime       0.50      0.75      0.60        59
     disaster       0.58      0.49      0.53        59
      economy       0.48      0.49      0.48        59
    education       0.55      0.73      0.63        59
environmental       0.50      0.66      0.57        58
       health       0.42      0.66      0.52        59
humaninterest       0.40      0.24      0.30        59
       labour       0.55      0.47      0.51        59
    lifestyle       0.36      0.40      0.38        58
        other       0.25      0.02      0.03        58
     politics       0.36      0.24      0.29        59
     religion       0.48      0.39      0.43        